In [2]:
from hdbcli import dbapi
import pandas as pd
import os

from hana_ml import dataframe
from dotenv import load_dotenv

from datetime import datetime, timezone

In [1]:
load_dotenv()
conn = dataframe.ConnectionContext(address='localhost', port=30015, user=os.getenv('hana_user'), password=os.getenv('hana_password'))

In [10]:
hdf_articles = conn.table('SAP_NEWS_CENTER_ARTICLES', schema=os.getenv('hana_nc_schema'))
hdf_articles.head(5).collect()

,ID,URL,SOURCE,PREMIUMCONTENT,CONTENT,CATEGORY,AUTHOR,PUBDATE,LASTMODIFIED,DESCRIPTION,LOCALE,_RSS,_FETCHTIME,HEADLINE,_EXTERNALID
0,00e5bbc4-a866-5219-91bb-66c0ab8e2785,https://www.theguardian.com/world/2021/mar/23/...,The Guardian,False,"Netanyahu’s Likud party is ahead in polls, but...",World news,None,2021-03-23 05:00:51,2021-03-23 05:02:26,"Netanyahu’s Likud party is ahead in polls, but...",en_GB,"{""title"": ""Israelis vote in fourth national el...",1616480314,Israelis vote in fourth national election in t...,https://www.theguardian.com/world/2021/mar/23/...
1,016eedd4-16fe-5c5d-b28a-fd206aaf3595,https://www.theguardian.com/world/audio/2021/m...,The Guardian,False,"On 23 March 2020, the UK prime minister, Boris...",World news,None,2021-03-23 05:00:52,2021-03-23 05:00:52,"A year on from the UK’s first lockdown, Ian Sa...",en_GB,"{""title"": ""Covid-19: what happens next? \u2013...",1616476697,Covid-19: what happens next? – podcast,https://www.theguardian.com/world/audio/2021/m...
2,01c5a68e-b15e-5571-88c4-abfd7466bc2c,https://www.theguardian.com/world/video/2021/m...,The Guardian,False,A north London business centre previously used...,World news,None,2021-03-11 21:21:45,2021-03-11 21:21:47,Patients arriving on its opening day expressed...,en_GB,"{""title"": ""'It's the right thing to do': Londo...",1615499527,'It's the right thing to do': Londoners receiv...,https://www.theguardian.com/world/video/2021/m...
3,02431ccb-8075-5c8a-b94b-710f71d7c651,https://www.theguardian.com/world/2021/mar/15/...,The Guardian,False,Move is first major rule change in months and ...,World news,None,2021-03-15 18:06:03,2021-03-15 18:11:20,Move is first major rule change in months and ...,en_GB,"{""title"": ""Portugal and Mauritius removed from...",1615832297,Portugal and Mauritius removed from England's ...,https://www.theguardian.com/world/2021/mar/15/...
4,027c2d97-6886-566b-b95b-b50d2c8e79f5,https://www.theguardian.com/world/2021/feb/25/...,The Guardian,False,CSIS included quote from A Perfect Spy in twee...,World news,None,2021-02-25 20:44:51,2021-02-25 23:31:21,CSIS included quote from A Perfect Spy in twee...,en_GB,"{""title"": ""Canada spy agency unwittingly seeks...",1614300779,Canada spy agency unwittingly seeks double age...,https://www.theguardian.com/world/2021/feb/25/...


In [8]:
hdf_articles.select('CONTENT', 'HEADLINE', 'SOURCE', 'PUBDATE', '_FETCHTIME', 'LOCALE', 'DESCRIPTION').head(5).collect()

,CONTENT,HEADLINE,SOURCE,PUBDATE,_FETCHTIME,LOCALE
0,"Netanyahu’s Likud party is ahead in polls, but...",Israelis vote in fourth national election in t...,The Guardian,2021-03-23 05:00:51,1616480314,en_GB
1,"On 23 March 2020, the UK prime minister, Boris...",Covid-19: what happens next? – podcast,The Guardian,2021-03-23 05:00:52,1616476697,en_GB
2,A north London business centre previously used...,'It's the right thing to do': Londoners receiv...,The Guardian,2021-03-11 21:21:45,1615499527,en_GB
3,Move is first major rule change in months and ...,Portugal and Mauritius removed from England's ...,The Guardian,2021-03-15 18:06:03,1615832297,en_GB
4,CSIS included quote from A Perfect Spy in twee...,Canada spy agency unwittingly seeks double age...,The Guardian,2021-02-25 20:44:51,1614300779,en_GB


In [9]:
hdf_articles.distinct('LOCALE').collect()

,LOCALE
0,de_DE
1,en_GB
2,en_US


In [11]:
df_articles = hdf_articles.filter("LOCALE = 'de_DE' AND CONTENT IS NOT NULL").select('CONTENT', 'HEADLINE', 'SOURCE', 'PUBDATE', '_FETCHTIME', 'LOCALE', 'DESCRIPTION').head(1000).collect()

In [2]:
connection = dbapi.connect(
            address='localhost',
            port=30015,
            user=os.getenv('hana_user'),
            password=os.getenv('hana_password'),
            currentSchema=os.getenv('hana_nc_schema'),
            connectTimeout=5000,
            communicationTimeout=60000
        )

cursor = connection.cursor()
cursor.setfetchsize(100000)

stmt = 'SELECT "CONTENT", "HEADLINE", "SOURCE", "PUBDATE", "_FETCHTIME", "LOCALE", "DESCRIPTION" FROM "SAP_NEWS_CENTER_ARTICLES" WHERE "LOCALE" = ? AND "CONTENT" is not null AND "DESCRIPTION" NOT LIKE ? AND "URL" NOT LIKE ? LIMIT 5000 OFFSET ?'

df_result = None
for i in range(0, 20):
    offset = i * 5000
    print(f'selecting with offset {offset}')
    cursor.execute(stmt, ('de_DE', '%Video mit Transkript%', '%welt%plus%', offset))

    cols = [
        col[1][0]
        for col in enumerate(cursor.description)
    ]
    result_rows = cursor.fetchall()

    if cursor.rowcount > 0:
        # Construct the data frame
        if df_result is None: 
            df_result = pd.DataFrame.from_records(result_rows, columns=cols)
        else:
            df_result = df_result.append(pd.DataFrame.from_records(result_rows, columns=cols))

df_result.shape

selecting with offset 0
selecting with offset 5000
selecting with offset 10000
selecting with offset 15000
selecting with offset 20000
selecting with offset 25000
selecting with offset 30000
selecting with offset 35000
selecting with offset 40000
selecting with offset 45000
selecting with offset 50000
selecting with offset 55000
selecting with offset 60000
selecting with offset 65000
selecting with offset 70000
selecting with offset 75000
selecting with offset 80000
selecting with offset 85000
selecting with offset 90000
selecting with offset 95000


(81094, 7)

In [19]:
df_result.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74868 entries, 0 to 4867
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CONTENT     74868 non-null  object        
 1   HEADLINE    74868 non-null  object        
 2   SOURCE      74868 non-null  object        
 3   PUBDATE     74868 non-null  datetime64[ns]
 4   _FETCHTIME  74868 non-null  int64         
 5   LOCALE      74868 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 461.9 MB


In [3]:
filename = 'export-articles-de-' + datetime.now(tz=timezone.utc).isoformat()
df_result.to_csv('../data/'+filename)

In [3]:
df = pd.read_csv('../data/export-articles-de-2021-09-08T14:45:27.515899+00:00')

In [22]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74868 entries, 0 to 74867
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  74868 non-null  int64 
 1   CONTENT     71155 non-null  object
 2   HEADLINE    74868 non-null  object
 3   SOURCE      74868 non-null  object
 4   PUBDATE     74868 non-null  object
 5   _FETCHTIME  74868 non-null  int64 
 6   LOCALE      74868 non-null  object
dtypes: int64(2), object(5)
memory usage: 466.7 MB


In [24]:
df['CONTENT'].isna().any()

True

In [25]:
df_result['CONTENT'].isna().any()

False

In [26]:
df[df['CONTENT'].isna()]

,Unnamed: 0,CONTENT,HEADLINE,SOURCE,PUBDATE,_FETCHTIME,LOCALE
7778,2778,NaN,Leserdiskussion: Wie bewerten Sie den Einfluss...,Sueddeutsche Zeitung,2021-04-08 13:54:07,1617890245,de_DE
7779,2779,NaN,Leserdiskussion: Coronavirus und Unternehmen -...,Sueddeutsche Zeitung,2021-04-08 13:54:02,1617890245,de_DE
7781,2781,NaN,Coronavirus in Deutschland: Intensivmediziner ...,Sueddeutsche Zeitung,2021-04-08 15:53:50,1617897518,de_DE
7783,2783,NaN,SZ-Umfrage: Wie geht es Dir?,Sueddeutsche Zeitung,2021-04-09 08:16:19,1617956705,de_DE
7784,2784,NaN,Corona-Maßnahmen: Bund will Gesetzentwurf für ...,Sueddeutsche Zeitung,2021-04-09 09:21:42,1617960350,de_DE
...,...,...,...,...,...,...,...
68012,3012,NaN,Gesundheit: Machen soziale Medien krank?,Sueddeutsche Zeitung,2021-04-28 08:30:40,1619598848,de_DE
68566,3566,NaN,Leserdiskussion: Soziale Ungleichheit: Sollte ...,Sueddeutsche Zeitung,2021-04-10 11:36:22,1618055339,de_DE
68569,3569,NaN,Fußball-Bayernliga: Akkurt beendet Karriere,Sueddeutsche Zeitung,2021-04-11 18:32:59,1618166523,de_DE
68576,3576,NaN,"SZ-Podcast: Wirecard: 1,9 Milliarden Lügen",Sueddeutsche Zeitung,2021-04-15 07:45:00,1618473462,de_DE


In [33]:
df_result

,CONTENT,HEADLINE,SOURCE,PUBDATE,_FETCHTIME,LOCALE
0,US-Präsident Joe Biden hat sich erneut gegen d...,US-Präsident Biden: Nord Stream 2 ist „kompliz...,Welt,2021-04-15 22:44:07,1618526736,de_DE
1,Die Grünen ziehen mit Annalena Baerbock als Ka...,Annalena Baerbock wird Kanzlerkandidatin der G...,Welt,2021-04-19 09:04:13,1618823447,de_DE
2,Mehr in Kürze,"Gesundheitsminister: Spahn rechnet damit, dass...",Welt,2021-04-22 11:07:16,1619089901,de_DE
3,Die erste Auslandsreise von US-Präsident Joe B...,Joe Biden: Erste Auslandsreise als US-Präsiden...,Welt,2021-04-23 16:20:13,1619195361,de_DE
4,Die Bundesregierung bietet Griechenland die Üb...,Rückführungen: Deutschland bietet Griechenland...,Welt,2021-04-24 21:59:57,1619301713,de_DE
...,...,...,...,...,...,...
4863,"Sie hat sich jetzt in ihre Villa in New Haven,...",USA: Neues von den Chubenfelds,Sueddeutsche Zeitung,2021-07-15 16:01:56,1626365321,de_DE
4864,Die Erschütterung des Bombenanschlags spüren s...,"Geschichte des LKA: ""Los, verschwindet, räumt ...",Sueddeutsche Zeitung,2021-07-15 16:01:54,1626365321,de_DE
4865,Weil die Wiesn auch in diesem Jahr wegen der C...,Freizeit und Kultur in München: Was in diesem ...,Sueddeutsche Zeitung,2021-07-15 15:59:56,1626365322,de_DE
4866,"Wo ist die Ministerpräsidentenkonferenz, wenn ...",Flutkatastrophe: Jetzt bräuchten wir die Minis...,Welt,2021-07-15 14:18:06,1626365323,de_DE


In [37]:
df_result.reset_index()

,index,CONTENT,HEADLINE,SOURCE,PUBDATE,_FETCHTIME,LOCALE
0,0,US-Präsident Joe Biden hat sich erneut gegen d...,US-Präsident Biden: Nord Stream 2 ist „kompliz...,Welt,2021-04-15 22:44:07,1618526736,de_DE
1,1,Die Grünen ziehen mit Annalena Baerbock als Ka...,Annalena Baerbock wird Kanzlerkandidatin der G...,Welt,2021-04-19 09:04:13,1618823447,de_DE
2,2,Mehr in Kürze,"Gesundheitsminister: Spahn rechnet damit, dass...",Welt,2021-04-22 11:07:16,1619089901,de_DE
3,3,Die erste Auslandsreise von US-Präsident Joe B...,Joe Biden: Erste Auslandsreise als US-Präsiden...,Welt,2021-04-23 16:20:13,1619195361,de_DE
4,4,Die Bundesregierung bietet Griechenland die Üb...,Rückführungen: Deutschland bietet Griechenland...,Welt,2021-04-24 21:59:57,1619301713,de_DE
...,...,...,...,...,...,...,...
74863,4863,"Sie hat sich jetzt in ihre Villa in New Haven,...",USA: Neues von den Chubenfelds,Sueddeutsche Zeitung,2021-07-15 16:01:56,1626365321,de_DE
74864,4864,Die Erschütterung des Bombenanschlags spüren s...,"Geschichte des LKA: ""Los, verschwindet, räumt ...",Sueddeutsche Zeitung,2021-07-15 16:01:54,1626365321,de_DE
74865,4865,Weil die Wiesn auch in diesem Jahr wegen der C...,Freizeit und Kultur in München: Was in diesem ...,Sueddeutsche Zeitung,2021-07-15 15:59:56,1626365322,de_DE
74866,4866,"Wo ist die Ministerpräsidentenkonferenz, wenn ...",Flutkatastrophe: Jetzt bräuchten wir die Minis...,Welt,2021-07-15 14:18:06,1626365323,de_DE


In [4]:
from langdetect import detect
df = df.dropna()

def detect_language(x):
    try:
        return detect(x)
    except:
        return None
df['lang_detected'] = df['CONTENT'].apply(lambda x: detect_language(x))

In [11]:
df['CONTENT'].isna().any()

False

# Get Cluster Data

In [2]:
clusters = conn.table('SAP_NEWS_CENTER_TOPICCLUSTERS', schema=os.getenv('hana_nc_schema'))
df_108 = clusters.filter("CLUSTER_LABEL = 108").select("START_DATE", "ARTICLE_ID").collect()

In [16]:
ids = ','.join(["'"+id+"'" for id in df_108['ARTICLE_ID'].tolist()])
df_108_articles = conn.table('SAP_NEWS_CENTER_ARTICLES', schema=os.getenv('hana_nc_schema')).filter(f"ID in ({ids})").collect()

In [17]:
cluster_content = df_108_articles.iloc[[1,2,3,4,5,8,12,13]]['CONTENT']

In [18]:
cluster_content

1     „Lass dich impfen, um deine Zukunft zu gestalt...
2     Der Vorsitzende der Ständigen Impfkommission (...
3     Brandenburgs Bildungsministerin Britta Ernst i...
4     STIKO-Mitglied Zepp hält eine begrenzte Empfeh...
5     Gesunde Kinder und Jugendliche haben nur ein s...
8     Milloni Doshi, a 25-year-old student from Indi...
12    Der 12-jährige Jeremy Haworth wurde im kalifor...
13    Der Vorsitzende der Ständigen Impfkommission (...
Name: CONTENT, dtype: object